## Prepare data reader for a given chain and date

In [1]:
from op_analytics.coreutils.duckdb_inmem import init_client
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.datapipeline.etl.intermediate.construct import construct_data_readers

from op_analytics.datapipeline.models.compute.udfs import create_duckdb_macros


# Define the input data range.
read_batches: list[DataReader] = construct_data_readers(
    chains=["op"],
    models=["event_emitting_transactions_list"],
    range_spec="@20241118:+1",
    read_from=DataLocation.GCS
)


# Select input for one date and build the intermediate model inputs.
batch = read_batches[0]


duckdb_client = init_client()
create_duckdb_macros(duckdb_client)


2024-12-13 07:19:14 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=25 process=12154
2024-12-13 07:19:14 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=12154
2024-12-13 07:19:14 [debug    ] loaded vault: 17 items         filename=vault.py lineno=76 process=12154
2024-12-13 07:19:15 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=37 process=12154
2024-12-13 07:19:15 [info     ] prepared 1 input batches.      filename=reader_bydate.py lineno=97 process=12154


## Run the model

This automatically registers the model outputs as duckdb tables.

In [2]:
from op_analytics.datapipeline.models.compute.testutils import execute_model_in_memory

execute_model_in_memory(
    duckdb_client=duckdb_client,
    model="event_emitting_transactions_list",
    data_reader=batch,
    limit_input_parquet_files=1
)

# The duckdb database will have the following:
#   - input tables
#   - views used by the model
#   - model outputs
# 
# You can use duckdb to inspect any of the above results.
duckdb_client.sql("SHOW TABLES")

2024-12-13 07:19:15 [info     ] Executing model...             filename=testutils.py lineno=220 process=12154
2024-12-13 07:19:15 [info     ] duckdb dataset='ingestion/logs_v1' using 1/22 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/logs_v1/chain=op/dt=2024-11-18/000128144000.parquet filename=reader.py lineno=68 process=12154
2024-12-13 07:19:16 [info     ] registered view: 'ingestion_logs_v1' using 1 parquet paths filename=client.py lineno=53 process=12154
2024-12-13 07:19:16 [info     ] duckdb dataset='ingestion/transactions_v1' using 1/22 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/transactions_v1/chain=op/dt=2024-11-18/000128144000.parquet filename=reader.py lineno=68 process=12154
2024-12-13 07:19:18 [info     ] registered view: 'ingestion_transactions_v1' using 1 parquet paths filename=client.py lineno=53 process=12154
2024-12-13 07:19:18 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=12154 templa

┌─────────────────────────────────────┐
│                name                 │
│               varchar               │
├─────────────────────────────────────┤
│ event_emitting_transactions_list    │
│ event_emitting_transactions_list_v1 │
│ ingestion_logs_v1                   │
│ ingestion_transactions_v1           │
└─────────────────────────────────────┘

## Verify model results

In [3]:
duckdb_client.sql("SELECT * FROM event_emitting_transactions_list_v1 LIMIT 10")

┌────────────┬─────────────────────┬─────────────────┬─────────┬─────────┬──────────┬──────────────┬────────────────────────────────────────────────────────────────────┬──────────────────┬────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┐
│     dt     │     block_hour      │ block_timestamp │ network │  chain  │ chain_id │ block_number │                          transaction_hash                          │ transaction_type │ count_total_events │ count_approval_events │ count_wrapping_events │ count_transfer_events │
│    date    │      timestamp      │     uint32      │ varchar │ varchar │  int32   │    int64     │                              varchar                               │      int32       │       int64        │     decimal(38,0)     │     decimal(38,0)     │     decimal(38,0)     │
├────────────┼─────────────────────┼─────────────────┼─────────┼─────────┼──────────┼──────────────┼──────────────────────────────────────────────────────

### Check the data output size

In [4]:
duckdb_client.sql("SELECT COUNT(*) AS interm_num_txs FROM event_emitting_transactions_list_v1")

┌────────────────┐
│ interm_num_txs │
│     int64      │
├────────────────┤
│          16679 │
└────────────────┘

In [5]:
duckdb_client.sql("SELECT COUNT(DISTINCT transaction_hash) as ingestion_num_txs FROM ingestion_logs_v1")

┌───────────────────┐
│ ingestion_num_txs │
│       int64       │
├───────────────────┤
│             16679 │
└───────────────────┘

In [6]:
duckdb_client.sql("SELECT topic0 FROM ingestion_logs_v1 LIMIT 10")


┌────────────────────────────────────────────────────────────────────┐
│                               topic0                               │
│                              varchar                               │
├────────────────────────────────────────────────────────────────────┤
│ 0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef │
│ 0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef │
│ 0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef │
│ 0x112c256902bf554b6ed882d2936687aaeb4225e8cd5b51303c90ca6cf43a8602 │
│ 0xcf2aa50876cdfbb541206f89af0ee78d44a2abf8d328e37fa4917f982149848a │
│ 0xb3e2773606abfd36b5bd91394b3a54d1398336c65005baf7bf7a05efeffaf75b │
│ 0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef │
│ 0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1 │
│ 0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822 │
│ 0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67 │
├─────

### You can also convert the results to dataframes to inspect them in more familiar ways

In [7]:
duckdb_client.sql("SELECT * FROM event_emitting_transactions_list_v1 LIMIT 5").pl()

dt,block_hour,block_timestamp,network,chain,chain_id,block_number,transaction_hash,transaction_type,count_total_events,count_approval_events,count_wrapping_events,count_transfer_events
date,datetime[μs],u32,str,str,i32,i64,str,i32,i64,"decimal[38,0]","decimal[38,0]","decimal[38,0]"
2024-11-18,2024-11-18 00:00:00,1731890755,"""mainnet""","""op""",10,128145989,"""0x0f3da35693644abe74388ab71888…",0,11,0,0,5
2024-11-18,2024-11-18 00:00:00,1731890755,"""mainnet""","""op""",10,128145989,"""0xc2a2f0930d2ac4e7dec64fd4ceba…",0,1,0,1,0
2024-11-18,2024-11-18 00:00:00,1731890755,"""mainnet""","""op""",10,128145989,"""0xd2964ea0788e9cac8791337a99f3…",2,10,1,0,1
2024-11-18,2024-11-18 00:00:00,1731890757,"""mainnet""","""op""",10,128145990,"""0x6b6aa9ee2a63a6e07504e13163a1…",0,10,3,0,4
2024-11-18,2024-11-18 00:00:00,1731890757,"""mainnet""","""op""",10,128145990,"""0x3669f9fe8fdaaffda96bffb7184f…",2,6,0,0,1


### Get table schema

In [8]:
duckdb_client.sql("DESCRIBE event_emitting_transactions_list_v1")


┌───────────────────────┬───────────────┬─────────┬─────────┬─────────┬─────────┐
│      column_name      │  column_type  │  null   │   key   │ default │  extra  │
│        varchar        │    varchar    │ varchar │ varchar │ varchar │ varchar │
├───────────────────────┼───────────────┼─────────┼─────────┼─────────┼─────────┤
│ dt                    │ DATE          │ YES     │ NULL    │ NULL    │ NULL    │
│ block_hour            │ TIMESTAMP     │ YES     │ NULL    │ NULL    │ NULL    │
│ block_timestamp       │ UINTEGER      │ YES     │ NULL    │ NULL    │ NULL    │
│ network               │ VARCHAR       │ YES     │ NULL    │ NULL    │ NULL    │
│ chain                 │ VARCHAR       │ YES     │ NULL    │ NULL    │ NULL    │
│ chain_id              │ INTEGER       │ YES     │ NULL    │ NULL    │ NULL    │
│ block_number          │ BIGINT        │ YES     │ NULL    │ NULL    │ NULL    │
│ transaction_hash      │ VARCHAR       │ YES     │ NULL    │ NULL    │ NULL    │
│ transaction_ty